In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")
example = """
My name is Wolfgang and I live in 12 Oak Street, Bristol, SW21 2UV. 
My file ID is 9297669.
I'm having trouble connecting to linkedin.com. My email Wolfgang.hollang@gmail.com
my password for linked in is rogueishyellow9291
anyway please sort this out as quickly as possible.
Thanks very much for your help.
"""
doc = nlp(example)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Wolfgang 12 20 PERSON
12 35 37 CARDINAL
Oak Street 38 48 LOC
Bristol 50 57 ORG
2UV 64 67 CARDINAL
9297669 84 91 DATE
rogueishyellow9291 205 223 NORP


In [12]:
from datasets import load_dataset
data = load_dataset("ai4privacy/pii-masking-200k",split='train')
examples = [data[i]['source_text'] for i in range(10000)]
char_count = [len(text) for text in examples]
word_count = [len(text.split(' ')) for text in examples]
import numpy as np
print('Average chars:', np.mean(char_count))
print('Average words:', np.mean(word_count))
print('sample_text:', examples[0])

Average chars: 172.3057
Average words: 24.9396
sample_text: A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it?


In [13]:
out = list(nlp.pipe(examples))

In [16]:
idx = 1999
print(examples[idx])
for ent in out[idx].ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Hello Mrs., as our new Liaison you will be working closely with our teachers from 122 Ayden Manor. Your key contact will be Madie who is based in Bourgogne-Franche-Comté. We laud your commitment to confidentiality, particularly with regards to our students' personal data, including 942-71-8827.
Liaison 23 30 PERSON
122 82 85 CARDINAL
Ayden 86 91 GPE
Madie 124 129 ORG
Bourgogne-Franche-Comté 146 169 ORG
942 283 286 CARDINAL


In [18]:
TP, FP = 0
for pred, target in zip(out, data[:10_000]):
    pred_ents = [ent.text for ent in pred.ents]
    target_ents = [mask['value'] for mask in target['privacy mask']]



{'source_text': "Hello Mrs., as our new Liaison you will be working closely with our teachers from 122 Ayden Manor. Your key contact will be Madie who is based in Bourgogne-Franche-Comté. We laud your commitment to confidentiality, particularly with regards to our students' personal data, including 942-71-8827.",
 'target_text': "Hello [PREFIX], as our new [JOBTYPE] you will be working closely with our teachers from [BUILDINGNUMBER] [STREET]. Your key contact will be [FIRSTNAME] who is based in [STATE]. We laud your commitment to confidentiality, particularly with regards to our students' personal data, including [SSN].",
 'privacy_mask': [{'value': 'Mrs.', 'start': 6, 'end': 10, 'label': 'PREFIX'},
  {'value': 'Liaison', 'start': 23, 'end': 30, 'label': 'JOBTYPE'},
  {'value': '122', 'start': 82, 'end': 85, 'label': 'BUILDINGNUMBER'},
  {'value': 'Ayden Manor', 'start': 86, 'end': 97, 'label': 'STREET'},
  {'value': 'Madie', 'start': 124, 'end': 129, 'label': 'FIRSTNAME'},
  {'value':